In [1]:
from google.cloud import bigquery
bq = bigquery.Client("cs-544-377118")
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

# use "gcloud auth application-default revoke" when not working
# gcloud auth application-default login --scopes=openid,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/drive.readonly

In [2]:
#q1
q1 = bq.query("""
    SELECT geo_id 
    FROM bigquery-public-data.geo_us_boundaries.counties
    WHERE county_name = "Dane"
    LIMIT 10
    """,job_config=no_cache)
q1s = q1.to_dataframe()

#print("The geo_id for Dane County is: " + strr)
str(q1s["geo_id"][0])

'55025'

In [3]:
#q2
q2 = bq.query("""
    SELECT state_fips_code, COUNT(*) AS count
    FROM bigquery-public-data.geo_us_boundaries.counties
    GROUP BY state_fips_code
    ORDER BY count desc
    LIMIT 5
    """,job_config=no_cache)
q2_dict = q2.to_dataframe()

dict(zip(q2_dict["state_fips_code"], q2_dict["count"]))

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [4]:
#q3
q3_1 = q1.total_bytes_billed / 1024**4 #TB for query
q3_2 = q2.total_bytes_billed / 1024**4 
#$5 per TB

q3_1 = q3_1 * 5
q3_2 = q3_2 * 5
query_dict = {'q1': q3_1, 'q2': q3_2}
query_dict

#For each of those queries, they cost around .005 cents

{'q1': 4.76837158203125e-05, 'q2': 4.76837158203125e-05}

In [5]:
#q4
dataset = bigquery.Dataset("cs-544-377118.p7")
bq.create_dataset(dataset, exists_ok=True)

for ds in bq.list_datasets("cs-544-377118"):
    dsid = (str((ds.dataset_id[-2:2])))  
    
([dsid])

['p7']

In [6]:
#config = bigquery.ExternalConfig("PARQUET")
#config.source_uris = ["gs://vm1-544/hdma-wi-2021.parquet"] 
#https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.parquet
#table = bigquery.Table("cs-544-377118.p7.hdma") 
#table.external_data_configuration=config
#bq.create_table(table,exists_ok=True)

config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
job = bq.load_table_from_uri(["gs://vm1-544/hdma-wi-2021.parquet"],
                             "cs-544-377118.p7.hdma", job_config=config)
job.result()

LoadJob<project=cs-544-377118, location=US, id=03d90d52-209e-4650-963f-c93395632121>

In [7]:
#q5
q5 =  bq.query("""
    SELECT county_name, COUNT(*) AS count
    FROM bigquery-public-data.geo_us_boundaries.counties
    INNER JOIN cs-544-377118.p7.hdma on county_code = county_fips_code
    GROUP BY county_name
    ORDER BY count desc
    LIMIT 10
    """)
q5 = q5.to_dataframe()

dict(zip(q5["county_name"], q5["count"]))

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [8]:
url = "https://drive.google.com/open?id=1e2qLPyxZ7s5ibMyEg7bxX2wWYAQD2ROAUcZuv8fqhnA"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(dataset.table("vacation"))
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [9]:
#q6
q6 = bq.query("""
    SELECT COUNT(*) AS count
    FROM cs-544-377118.p7.vacation
    WHERE income = 50000
    GROUP BY income
    """)
q6 = q6.to_dataframe()

q6["count"][0]

2

In [10]:
#q7
q7 = bq.query("""
    SELECT county_name, COUNT(*) AS count
    FROM bigquery-public-data.geo_us_boundaries.counties, cs-544-377118.p7.vacation
    WHERE ST_DWithin(
         county_geom,
         ST_GeogPoint(longitude, latitude), 100) and state_fips_code = '55'
    GROUP BY county_name
    ORDER BY count desc
    """)
q7 = q7.to_dataframe()

dict(zip(q7["county_name"], q7["count"]))

{'Dane': 32,
 'Door': 5,
 'Jefferson': 4,
 'Brown': 3,
 'Bayfield': 3,
 'Walworth': 2,
 'Columbia': 2,
 'Milwaukee': 2,
 'Sheboygan': 1,
 'Monroe': 1,
 'La Crosse': 1,
 'Iowa': 1,
 'Dodge': 1,
 'Iron': 1,
 'Kewaunee': 1,
 'Adams': 1,
 'Waukesha': 1,
 'Green Lake': 1,
 'Oneida': 1,
 'Sauk': 1,
 'Barron': 1,
 'Ashland': 1,
 'Washburn': 1}

In [11]:
#q8
q8 = bq.query("""
CREATE OR REPLACE MODEL `cs-544-377118.p7.hdma-model`
OPTIONS (model_type="linear_reg",
  input_label_cols=["loan_amount"]) 
AS
SELECT income,loan_term, loan_amount
FROM `cs-544-377118.p7.hdma`
  """)
q8.to_dataframe()


q8_m = bq.query("""
SELECT *
FROM ML.EVALUATE(MODEL `cs-544-377118.p7.hdma-model`)
  """)
q8m = q8_m.to_dataframe()


#print("The model's r2 score is: ", q8m["r2_score"][0])
q8m["r2_score"][0]

0.2904647351471993

In [12]:
#q9
q9 = bq.query("""
SELECT *
FROM ML.WEIGHTS(MODEL `cs-544-377118.p7.hdma-model`)
    """)
q9 = q9.to_dataframe()

#print("The coefficient weight on income column:", q9["weight"][0])
q9["weight"][0]

0.8057773037176693

In [13]:
#q10
q10 = bq.query("""
SELECT COUNT(*) AS total_num_of_applications
FROM
  ML.PREDICT(MODEL `cs-544-377118.p7.hdma-model`,
    (
    SELECT income, loan_amount, 360 AS loan_term
    FROM `cs-544-377118.p7.vacation`))
""").to_dataframe()


q10p2 = bq.query("""
SELECT COUNT(*) AS total_num_with_higher_amts_than_model
FROM
  ML.PREDICT(MODEL `cs-544-377118.p7.hdma-model`,
    (
    SELECT loan_amount, income, 360 AS loan_term
    FROM `cs-544-377118.p7.vacation`
    WHERE loan_amount > income))
""").to_dataframe()
ratio = q10p2["total_num_with_higher_amts_than_model"][0] / q10["total_num_of_applications"][0]

#print("The ratio:", ratio)
ratio

0.7666666666666667